In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset
from ei_effort import Optimal_Effort
from ei_utils import *
from ei_model_dev import FairBatch

In [2]:
dataset = SyntheticDataset(seed=0)

In [3]:
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

model_params = torch.load('../ei_model.pkl')
model = LR(train_dataset.X.shape[1])
model.load_state_dict(model_params)

for module in model.layers:
    if hasattr(module, 'weight'):
        weights_0 = module.weight.data
    if hasattr(module, 'bias'):
        bias_0 = module.bias.data

theta_0 = torch.cat((weights_0[0], bias_0), 0)
theta_0

tensor([ 0.2218,  0.5095, -0.1942,  0.3345])

In [4]:
alpha = 0.1
tau = 0.5
thetas = torch.from_numpy(generate_grid(theta_0.numpy(), alpha, n=15, ord=np.inf)).float()
effort_model = Optimal_Effort(tau)

In [14]:
Y_hat = model(test_dataset.X).reshape(-1).detach().numpy()
X = test_dataset.X[(Y_hat<tau),:]
Z = test_dataset.Z[(Y_hat<tau)]
X_hat_max = effort_model(model, test_dataset, X)

losses = []
loss_fn = torch.nn.BCELoss(reduction = 'mean')

for theta in tqdm(thetas):
    weights, bias = theta[:-1].clone().reshape(1, -1), theta[[-1]].clone()

    model_adv = deepcopy(model)
    for module in model_adv.layers:
        if hasattr(module, 'weight'):
            module.weight.data = weights.float()
        if hasattr(module, 'bias'):
            module.bias.data = bias.float()
            
    Y_hat_max = model_adv(X_hat_max).reshape(-1)
    
    fair_loss = 0.
    loss_mean = loss_fn(Y_hat_max, torch.ones(len(Y_hat_max)))
    loss_z = torch.zeros(len(dataset.sensitive_attrs))
    for z in test_dataset.sensitive_attrs:
        z = int(z)
        group_idx = (Z == z)
        if group_idx.sum() == 0:
            continue
        loss_z[z] = loss_fn(Y_hat_max[group_idx], torch.ones(group_idx.sum()))
        fair_loss += torch.abs(loss_z[z] - loss_mean)
    
    losses.append(fair_loss.detach().item())

 38%|███▊      | 14680/38416 [00:04<00:06, 3561.04it/s]

In [13]:
max_i = np.argmax(losses)
fair_loss = losses[max_i]
theta_r = thetas[max_i]
weights, bias = theta_r[:-1].clone().reshape(1, -1), theta_r[[-1]].clone()

model_adv = deepcopy(model)
for module in model_adv.layers:
    if hasattr(module, 'weight'):
        module.weight.data = weights.float()
    if hasattr(module, 'bias'):
        module.bias.data = bias.float()
        
X_hat_max = effort_model(model_adv, test_dataset, test_dataset.X)
Y_hat_max = model_adv(X_hat_max).reshape(-1).detach().numpy()

accuracy, ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)

In [12]:
alphas = (theta_r-theta_0).abs()
print(f'alpha               |   {alpha}')
print(f'Accuracy            |   {accuracy:.3f}')
print(f'Fairness Loss       |   {fair_loss:.3f}')
print(f'EI Disparity        |   {ei_disparity:.3f}')
print(f'theta_0             |   {theta_0}')
print(f'theta_r             |   {theta_r}')
print(f'alphas              |   {np.round(alphas,4)}')

alpha               |   0.1
Accuracy            |   0.608
Fairness Loss       |   0.079
EI Disparity        |   0.103
theta_0             |   tensor([ 0.2218,  0.5095, -0.1942,  0.3345])
theta_r             |   tensor([ 0.3075,  0.5952, -0.1085,  0.2345])
alphas              |   tensor([0.0857, 0.0857, 0.0857, 0.1000])


In [8]:
ei_model = FairBatch(model, effort_model, 1e-7)
pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict_r(test_dataset, alpha)

pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)

for module in ei_model.model_adv.layers:
    if hasattr(module, 'weight'):
        pga_weights_r = module.weight.data
    if hasattr(module, 'bias'):
        pga_bias_r = module.bias.data
pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

In [9]:
alphas = (pga_theta_r-theta_0).abs()
print('PGA Results')
print(f'alpha               |   {alpha}')
print(f'Accuracy            |   {pga_accuracy:.3f}')
print(f'Fairness Loss       |   {pga_fair_loss:.3f}')
print(f'EI Disparity        |   {pga_ei_disparity:.3f}')
print(f'theta_0             |   {theta_0}')
print(f'theta_r             |   {pga_theta_r}')
print(f'alphas              |   {np.round(alphas,4)}')

PGA Results
alpha               |   0.1
Accuracy            |   0.608
Fairness Loss       |   0.089
EI Disparity        |   0.113
theta_0             |   tensor([ 0.2218,  0.5095, -0.1942,  0.3345])
theta_r             |   tensor([ 0.3218,  0.6095, -0.0942,  0.2345])
alphas              |   tensor([0.1000, 0.1000, 0.1000, 0.1000])
